# Summary

This script is to processing the mass spectral embeddings into arrays and connect them to the class labels.



In [1]:
import os
import gensim
import pickle
import time
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from spec2vec.vector_operations import calc_vector
from collections import OrderedDict, Counter

RDKit WARNING: [22:07:15] Enabling RDKit 2019.09.3 jupyter extensions


## Import data from file

In [2]:
data_path = "/mnt/LTR_userdata/hooft001/mass_spectral_embeddings/datasets/ALL_GNPS_210409_positive/"
base = "ALL_GNPS_210409_positive_cleaned"
spectra_data = "_peaks_processed_s2v_only_annotated.pickle"

embedding_path = "/mnt/LTR_userdata/hooft001/mass_spectral_embeddings/embeddings/ALL_GNPS_210409_positive/ALL_GNPS_210409_positive_cleaned_spec2vec_embedding_iter_15.model"
save_path = "/mnt/scratch/ding013/MS2ChemClass/hooft_data/embedding_visualization_out/"

spectrum_file = os.path.join(data_path, base+".pickle")
os.path.exists(spectrum_file)

processed_spectrums_file = os.path.join(data_path, base+spectra_data)
with open(processed_spectrums_file, 'rb') as inf:
    spectrums_processed = pickle.load(inf)

len(spectrums_processed)

# TEST DATA
# spectrums_processed = spectrums_processed[1:100]
# len(spectrums_processed)


# ### Keep annotated spectra
# Annotated with inchikey

annot_spectrums_processed = []
for spec in spectrums_processed:
    inchikey = spec.metadata.get("inchikey")
    if inchikey:
        annot_spectrums_processed.append(spec)
len(annot_spectrums_processed)

spectrums_ids = [s.metadata.get("spectrum_id") for s in annot_spectrums_processed]


any([s==None for s in spectrums_ids])

False

In [3]:
print(spectrums_processed[1])

## Create mass spectral embedding vectors

In [4]:
# ### Create spectral embedding vectors


# import model
model_file = os.path.join(embedding_path)
model = gensim.models.Word2Vec.load(model_file)


from spec2vec import SpectrumDocument

documents_spectrums_processed = [SpectrumDocument(s, n_decimals=2) for s in annot_spectrums_processed]
print(len(documents_spectrums_processed))


spectrum_vectors= [calc_vector(model, s, intensity_weighting_power=0.5) for s in documents_spectrums_processed]

spectrum_vectors = np.vstack(spectrum_vectors)
spectrum_vectors.shape

# and for UniqueInchikey
sys.path.insert(0, os.path.join(os.path.split(os.getcwd()[:-1])[0], "scripts"))

from mass_differences.processing import get_ids_for_unique_inchikeys ## This function is cited from Joris Louwen
uniq_ids = get_ids_for_unique_inchikeys(annot_spectrums_processed)


spectrum_vectors_unique = [spectrum_vectors[i] for i in uniq_ids]
spectrum_vectors_unique = np.vstack(spectrum_vectors_unique)
spectrum_vectors_unique.shape

187152


(16360, 300)

## Link classification labels to embeddings

In [5]:
# ### Link classification labels to selected spectra

classifications_path = "/mnt/LTR_userdata/hooft001/mass_spectral_embeddings/classifications/ALL_GNPS_210409_positive/"
classes_file = os.path.join(classifications_path, "ALL_GNPS_210409_positive_processed_annotated_CF_NPC_classes.txt")
classes = {}  # inchikey: [classes] in order of header
with open(classes_file) as inf:
    header = inf.readline().strip().split('\t')
    for line in inf:
        line = line.strip().split('\t')
        classes[line[0]] = line[2:]
print(f'read classes for {len(classes)} inchikeys')
print("\nclasses order:")
print(header)

spectra_classes = []
empty_class = ["" for _ in range(len(header)-2)]
assert len(empty_class) == len(header)-2
for s_id, spec in zip(spectrums_ids, annot_spectrums_processed):
    inchikey = spec.metadata.get("inchikey")
    s_classes = classes.get(inchikey, empty_class)
    assert len(s_classes) == len(header)-2
    res = [s_id] + s_classes
    spectra_classes.append(res)
print(len(spectra_classes))
print(spectra_classes[:2])

new_header = ['spectrum_id'] + header[2:]
assert len(new_header) == len(spectra_classes[0])
print(new_header)

read classes for 19824 inchikeys

classes order:
['inchi_key', 'smiles', 'cf_kingdom', 'cf_superclass', 'cf_class', 'cf_subclass', 'cf_direct_parent', 'npc_class_results', 'npc_superclass_results', 'npc_pathway_results', 'npc_isglycoside']
187152
[['CCMSLIB00000001547', 'Organic compounds', 'Organic acids and derivatives', 'Peptidomimetics', 'Hybrid peptides', 'Hybrid peptides', 'Cyclic peptides; Microcystins', 'Oligopeptides', 'Amino acids and Peptides', '0'], ['CCMSLIB00000001548', 'Organic compounds', 'Organic acids and derivatives', 'Peptidomimetics', 'Depsipeptides', 'Cyclic depsipeptides', 'Cyclic peptides', 'Oligopeptides', 'Amino acids and Peptides', '0']]
['spectrum_id', 'cf_kingdom', 'cf_superclass', 'cf_class', 'cf_subclass', 'cf_direct_parent', 'npc_class_results', 'npc_superclass_results', 'npc_pathway_results', 'npc_isglycoside']


In [6]:
spectra_classes[0]

['CCMSLIB00000001547',
 'Organic compounds',
 'Organic acids and derivatives',
 'Peptidomimetics',
 'Hybrid peptides',
 'Hybrid peptides',
 'Cyclic peptides; Microcystins',
 'Oligopeptides',
 'Amino acids and Peptides',
 '0']

### Get the ordered classification labels

In [44]:
# i_chosen = -4  # npc_class # choose which class (npc/cf) to plot (in new_header)
# i_chosen = 3 # cf_class

## Collect the ordered labels in all classes
all_classes_ori = []
for lbs in range(1,len(new_header)):
    #print(new_header[lbs])
    chosen_classes_ori = []
    uncl = "Unclassified"
    uncl_other = "Other/Unclassified"
    add_uncl = False
    l=0
    for s_cl in spectra_classes:
        chosen_class = s_cl[lbs]
        
        if not chosen_class:
            #print(chosen_class)
            chosen_class = uncl
        else:  # separate in single classes - there are compounds with 3 superclasses for example
            chosen_class_lbs = chosen_class.split(";")
            if len(chosen_class_lbs) >=2:
                l+=1
            chosen_class = chosen_class_lbs[0]
        chosen_classes_ori.append(chosen_class)
    len(chosen_classes_ori)
    all_classes_ori.append(chosen_classes_ori)
    print(new_header[lbs]+"includes "+str(l)+" multi-label groups")
len(all_classes_ori)

cf_kingdomincludes 0 multi-label groups
cf_superclassincludes 0 multi-label groups
cf_classincludes 0 multi-label groups
cf_subclassincludes 0 multi-label groups
cf_direct_parentincludes 0 multi-label groups
npc_class_resultsincludes 15176 multi-label groups
npc_superclass_resultsincludes 5199 multi-label groups
npc_pathway_resultsincludes 6659 multi-label groups
npc_isglycosideincludes 0 multi-label groups


9

In [48]:
## Collect the orderen labels with unique inchikey in all classes
all_classes_ori_ui = []
for k in all_classes_ori:
    chosen_classes_ori_ui = [k[i] for i in uniq_ids]
    len(chosen_classes_ori_ui)
    all_classes_ori_ui.append(chosen_classes_ori_ui)
len(all_classes_ori_ui)

9

## Get the belonging relationship among labels in differnt class levels

In [51]:
cf_labels = all_classes_ori_ui[1:4] # Superclass, Class, Subclass
npc_labels = all_classes_ori_ui[5:8] # Class, SuperClass, Pathway

In [52]:
def get_belonging_relationship(superclass_lbs,class_lbs,subclass_lbs):
    class_belong_dict = {}
    subclass_belong_dict = {}
    for i,v in enumerate(subclass_lbs):
        subclass_belong_dict[v] = [class_lbs[i],superclass_lbs[i]]
    for i,v in enumerate(class_lbs):
        class_belong_dict[v] = superclass_lbs[i]
    return class_belong_dict, subclass_belong_dict

In [53]:
# For ClassyFire
cf_class_belong_dict, cf_subclass_belong_dict = get_belonging_relationship(cf_labels[0], cf_labels[1],cf_labels[2])
with open('/mnt/scratch/ding013/MS2ChemClass/embedding_arrays_for_classification/cf_class_belong.pickle', 'wb') as f:
    pickle.dump(cf_class_belong_dict, f)
with open('/mnt/scratch/ding013/MS2ChemClass/embedding_arrays_for_classification/cf_subclass_belong.pickle', 'wb') as f:
    pickle.dump(cf_subclass_belong_dict, f)

    # For NPClassifier
npc_superclass_belong_dict, npc_class_belong_dict = get_belonging_relationship(npc_labels[2],npc_labels[1],npc_labels[0])
with open('/mnt/scratch/ding013/MS2ChemClass/embedding_arrays_for_classification/npc_superclass_belong.pickle', 'wb') as f:
    pickle.dump(npc_superclass_belong_dict, f)
with open('/mnt/scratch/ding013/MS2ChemClass/embedding_arrays_for_classification/npc_class_belong.pickle', 'wb') as f:
    pickle.dump(npc_class_belong_dict, f)


In [54]:
print(len(cf_class_belong_dict))
print(len(cf_subclass_belong_dict))
print(len(npc_superclass_belong_dict))
print(len(npc_class_belong_dict))
print(len(cf_labels[0]))
#print(cf_subclass_belong_dict)

279
466
71
450
16360


## Save array as csv file

In [60]:
## Save the spectral embeddings array with unique inchikey

X_embedding = np.array(spectrum_vectors_unique)
np.savetxt('/mnt/scratch/ding013/MS2ChemClass/embedding_arrays_for_classification/X_spectral_embeddings.csv', X_embedding, delimiter=',')

In [61]:
## Save the ordered class labels array with unique inchikey
for i in range(len(all_classes_ori_ui)):
    y_label = all_classes_ori_ui[i]
    print(new_header[i+1])
    filepath_labels='/mnt/scratch/ding013/MS2ChemClass/embedding_arrays_for_classification/y_%s_labels.pickle' % new_header[i+1]
    with open(filepath_labels, 'wb') as f:
        pickle.dump(y_label,f)

cf_kingdom
cf_superclass
cf_class
cf_subclass
cf_direct_parent
npc_class_results
npc_superclass_results
npc_pathway_results
npc_isglycoside


In [62]:
# Save all class labels information
with open('/mnt/scratch/ding013/MS2ChemClass/embedding_arrays_for_classification/all_y_classifications.pickle','wb') as f:
    pickle.dump(all_classes_ori_ui,f)